In [1]:
import psycopg2
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, avg
from sklearn.mixture import GaussianMixture
import pandas as pd
import time
from psycopg2.extras import RealDictCursor



In [2]:
conn = psycopg2.connect(
    host = "holomentor-db-1.cn2osckgs62c.ap-southeast-1.rds.amazonaws.com",
    database = "holomentor_db",
    user = "postgres",
    password = "Holomentor123"
)

In [3]:
from datetime import datetime
import json

class DateTimeEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, datetime):
            return o.isoformat()  # Converts datetime to ISO format
        return super().default(o)

In [4]:
import json

def lambda_handler(conn):
    cur = conn.cursor()
    cur.execute("SELECT * FROM institute_class_students")
    records = cur.fetchall()
    
    # Get column names from the cursor description
    columns = [desc[0] for desc in cur.description]
    return records, columns

records, columns = lambda_handler(conn)

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, NullType
import datetime

spark = SparkSession.builder.appName("ReadFromDatabase").getOrCreate()

df = spark.createDataFrame(records, schema=StructType ([
    StructField("id", IntegerType(), True),
    StructField("student_id", IntegerType(), True),
    StructField("institute_id", IntegerType(), True),
    StructField("institute_student_id", IntegerType(), True),
    StructField("class_id", IntegerType(), True),
    StructField("registration_number", StringType(), True),
    StructField("created_at", TimestampType(), True),
    StructField("tier", StringType(), True),
    StructField("tier_last_updated_at", TimestampType(), True)
]))

# Show DataFrame
df.show()

KeyboardInterrupt: 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, current_timestamp, when
import psycopg2
import os

spark = SparkSession.builder.appName("UpdateSelectedColumnsByStudentID").getOrCreate()


update_data = [(4, 1), (14, 2)]

# Create a DataFrame from the update_data
tier_df = spark.createDataFrame(update_data, ["student_id", "tier"])

# Show DataFrame
tier_df.show()

+----------+----+
|student_id|tier|
+----------+----+
|         4|   1|
|        14|   2|
+----------+----+



In [ ]:
from pyspark.sql import functions as F
from datetime import datetime

updated_df = (
    df.join(tier_df.withColumnRenamed("tier", "new_tier"), on="student_id", how="left")
      .withColumn("tier", F.coalesce("new_tier", df["tier"].cast("string")))
      .withColumn("tier_last_updated_at", F.current_timestamp())
      .drop("new_tier") 
)


updated_df.show()

+----------+---+------------+--------------------+--------+-------------------+--------------------+----+--------------------+
|student_id| id|institute_id|institute_student_id|class_id|registration_number|          created_at|tier|tier_last_updated_at|
+----------+---+------------+--------------------+--------+-------------------+--------------------+----+--------------------+
|         4|  1|           1|                   1|       1|                  1|2024-10-16 19:31:...|   1|2024-11-09 20:08:...|
|        14|  3|           1|                  15|       1|               2332|2024-11-02 17:55:...|   2|2024-11-09 20:08:...|
+----------+---+------------+--------------------+--------+-------------------+--------------------+----+--------------------+



In [ ]:
def update_database_with_spark(conn, spark_df):
    # Assuming conn is the database connection (e.g., psycopg2 connection)
    cur = conn.cursor()

    # Convert Spark DataFrame to Pandas DataFrame
    update_data = spark_df.toPandas()

    # Loop through each record in the Pandas DataFrame
    for index, row in update_data.iterrows():
        student_id = row['student_id']
        tier = row['tier']
        
        # Prepare the SQL query to update the tier and tier_last_updated_at fields
        update_query = """
        UPDATE institute_class_students
        SET tier = %s, tier_last_updated_at = %s
        WHERE student_id = %s
        """
        
        current_time = datetime.now()
        # Execute the update query with the parameters
        cur.execute(update_query, (tier, current_time, student_id))

    # Commit the changes to the database
    conn.commit()
    cur.close()

update_database_with_spark(conn, updated_df)